In [1]:
import numpy as np
import string

In [2]:
np.random.seed(1234)

In [3]:
def remove_punctuation(s):
    return s.translate(str.maketrans('','',string.punctuation))

In [4]:
def add2dict(d,k,v):
    if k not in d:
        d[k] = []
    d[k].append(v)

In [5]:
def getlinesfromtextfile(filename):
    lines = []
    for line in open(filename):
        lines.append(line)
    return lines

In [6]:
def markov_second_order(lines):
    initial = {} # dictionary of pharases
    first_order_markov = {} # second word only.
    second_order_markov = {} # from second word onwards
    for line in lines:
        tokens = remove_punctuation(line.rstrip().lower()).split()
        T = len(tokens)
        for i in range(T):
            t = tokens[i]
            if i ==0:
                # first word
                initial[t] = initial.get(t,0.) + 1
            else:
                t_1 = tokens[i -1]
                if i == T-1:
                    # last word
                    add2dict(second_order_markov, (t_1,t),'END')
                if i == 1:
                    # second word
                    add2dict(first_order_markov,t_1,t)
                else:
                    # all the remaining.
                    t_2 = tokens[i-2]
                    add2dict(second_order_markov, (t_2,t_1),t)
    return initial,first_order_markov,second_order_markov

In [7]:
# normalize distribution
def normalize_initial_distribution(initial):
    initial_total = sum(initial.values())
    for t,c in initial.items():
        initial[t] = c/initial_total

def normalize_markov_order_distribution(ts):
    d = {}
    n = len(ts)
    for t in ts:
        d[t] = d.get(t,0.) + 1
    for t,c in d.items():
        d[t] = c/n
    return d

In [8]:
# test with 2 lines
test_data = getlinesfromtextfile('robert_frost.txt')[:4]
test_data

['Two roads diverged in a yellow wood,\n',
 'And sorry I could not travel both\n',
 'And be one traveler, long I stood\n',
 'And looked down one as far as I could\n']

In [9]:
initial,first_order_markov,second_order_markov = markov_second_order(test_data)
print('initial\n',initial)
print('first_order_markov\n',first_order_markov)
print('second_order_markov\n',second_order_markov)

initial
 {'two': 1.0, 'and': 3.0}
first_order_markov
 {'two': ['roads'], 'and': ['sorry', 'be', 'looked']}
second_order_markov
 {('two', 'roads'): ['diverged'], ('roads', 'diverged'): ['in'], ('diverged', 'in'): ['a'], ('in', 'a'): ['yellow'], ('yellow', 'wood'): ['END'], ('a', 'yellow'): ['wood'], ('and', 'sorry'): ['i'], ('sorry', 'i'): ['could'], ('i', 'could'): ['not', 'END'], ('could', 'not'): ['travel'], ('travel', 'both'): ['END'], ('not', 'travel'): ['both'], ('and', 'be'): ['one'], ('be', 'one'): ['traveler'], ('one', 'traveler'): ['long'], ('traveler', 'long'): ['i'], ('i', 'stood'): ['END'], ('long', 'i'): ['stood'], ('and', 'looked'): ['down'], ('looked', 'down'): ['one'], ('down', 'one'): ['as'], ('one', 'as'): ['far'], ('as', 'far'): ['as'], ('far', 'as'): ['i'], ('as', 'i'): ['could']}


In [10]:
def normalize(initial,first_order,second_order):
    normalize_initial_distribution(initial)
    for t_1,ts in first_order.items():
        first_order[t_1] = normalize_markov_order_distribution(ts)
    for t_1,ts in second_order.items():
        second_order[t_1] = normalize_markov_order_distribution(ts)

normalize(initial,first_order_markov,second_order_markov)
print('test_data\n',test_data)
print('initial\n',initial)
print('first_order_markov\n',first_order_markov)
print('second_order_markov\n',second_order_markov)

test_data
 ['Two roads diverged in a yellow wood,\n', 'And sorry I could not travel both\n', 'And be one traveler, long I stood\n', 'And looked down one as far as I could\n']
initial
 {'two': 0.25, 'and': 0.75}
first_order_markov
 {'two': {'roads': 1.0}, 'and': {'sorry': 0.3333333333333333, 'be': 0.3333333333333333, 'looked': 0.3333333333333333}}
second_order_markov
 {('two', 'roads'): {'diverged': 1.0}, ('roads', 'diverged'): {'in': 1.0}, ('diverged', 'in'): {'a': 1.0}, ('in', 'a'): {'yellow': 1.0}, ('yellow', 'wood'): {'END': 1.0}, ('a', 'yellow'): {'wood': 1.0}, ('and', 'sorry'): {'i': 1.0}, ('sorry', 'i'): {'could': 1.0}, ('i', 'could'): {'not': 0.5, 'END': 0.5}, ('could', 'not'): {'travel': 1.0}, ('travel', 'both'): {'END': 1.0}, ('not', 'travel'): {'both': 1.0}, ('and', 'be'): {'one': 1.0}, ('be', 'one'): {'traveler': 1.0}, ('one', 'traveler'): {'long': 1.0}, ('traveler', 'long'): {'i': 1.0}, ('i', 'stood'): {'END': 1.0}, ('long', 'i'): {'stood': 1.0}, ('and', 'looked'): {'down':

In [11]:
def sample_word(d):
    p0 = np.random.random()
    cumulative = 0
    for t,p in d.items():
        cumulative += p
        if p0 < cumulative:
            return t
    assert(False)

In [12]:
def generate():
    for i in range(2):
        sentence = []
        
        # start word
        w0 = sample_word(initial)
        sentence.append(w0)
        
        # sample second word
        w1 = sample_word(first_order_markov[w0])
        sentence.append(w1)
        
        # generate second order till end
        while True:
            w2 = sample_word(second_order_markov[w0,w1])
            if w2 == 'END':
                break
            sentence.append(w2)
            w0 = w1
            w1 = w2
        print(' '.join(sentence))

In [13]:
generate()

two roads diverged in a yellow wood
and looked down one as far as i could not travel both
